### imports and preparation

- place this notebook in ../typeform 
- store your own token for tf API in a file called "typeform_token.py" and save it locally (do not commit) in /typeform
- The file contains only one line: TOKEN="your_token"

In [5]:
import requests
import pandas as pd
import time
import textwrap

#import my own function
%run one_entry_to_dictionary.py
%run typeform_token.py

In [6]:
import sys
sys.path.append('../')

# 1. Pull Data from Typeform

### Connect to TF, see metainfo for form

In [7]:
#IMPORT TOKEN FILE 
auth_token= TOKEN
head = {'Authorization': 'Bearer ' + auth_token}
url = 'https://api.typeform.com/forms/OHziDn'

metainfo_form = requests.get(url, headers=head)
#response 200 means its good !
print(metainfo_form)
print(metainfo_form.json())

<Response [200]>
{'id': 'OHziDn', 'title': 'UN Women COVID-19 Survey', 'theme': {'href': 'https://api.typeform.com/themes/UTPtTC'}, 'workspace': {'href': 'https://api.typeform.com/workspaces/y2fgNQ'}, 'settings': {'is_public': True, 'is_trial': False, 'language': 'en', 'progress_bar': 'percentage', 'show_progress_bar': False, 'show_typeform_branding': False, 'meta': {'allow_indexing': False, 'description': "Help us understand women's experience during COVID-19.", 'image': {'href': 'https://images.typeform.com/images/r8wQiuyGeEu2'}}}, 'welcome_screens': [{'ref': '427323db-ebf3-48b6-8cf0-2f2ecdcebd7f', 'title': "Tell us how you're doing during lockdown with a few short questions, so we can make sure your voice is heard.\n\nWe'll use your answers to show the government and leaders what changes need to happen.\n\nThank you so much for being part of this.", 'properties': {'show_button': True, 'button_text': 'Start'}, 'attachment': {'type': 'image', 'href': 'https://images.typeform.com/image

### Pull all responses exclusing mail from Typeform

In [8]:
# get all answers but not emails
SIZE = "500"
url = "https://api.typeform.com/forms/OHziDn/responses?fields=Xa2T0bcZ5VnY,QUtA6k0gcrZf,ZNZyNVwMtwID,azlKRkvdcc7O,Vrzj2uv0i29C,Xh2W1d6FhXyK,t9EaTsONnJfR,OaujoVSY6NaS,ASolp4v3adZK,erHBE6gKxycf,ASQmq4h3vxRt,KEGztC8iYfyy,kWb7LUjdJyhb,g9goY6hIxmC9,jG2wzvAmOWxq,HNqA7BK7ZDFl&page_size="+SIZE

response = requests.get(url, headers=head)
#response 200 means its good !
print(response)

responses = response.json()

<Response [200]>


In [9]:
print("number of responses",responses['total_items'])
#look at the first response
print("the last response:",responses["items"][0])

number of responses 622
the last response: {'landing_id': 'lmykbgpt6c7v8zwulmykvg179i1hq716', 'token': 'lmykbgpt6c7v8zwulmykvg179i1hq716', 'response_id': 'lmykbgpt6c7v8zwulmykvg179i1hq716', 'landed_at': '2020-05-26T03:18:44Z', 'submitted_at': '2020-05-26T03:23:05Z', 'metadata': {'user_agent': 'Mozilla/5.0 (Linux; Android 10; SAMSUNG SM-G985F) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/11.2 Chrome/75.0.3770.143 Mobile Safari/537.36', 'platform': 'mobile', 'referer': 'https://unwomenuk.typeform.com/to/OHziDn', 'network_id': 'c823e07581', 'browser': 'touch'}, 'calculated': {'score': 0}, 'answers': [{'field': {'id': 'Xa2T0bcZ5VnY', 'ref': '49c4acab-fed2-46c2-804d-8f36db71a6d5', 'type': 'multiple_choice'}, 'type': 'choice', 'choice': {'id': 'P15fmXrpoRxj', 'label': 'Woman'}}, {'field': {'id': 'QUtA6k0gcrZf', 'ref': '81c603ce-bdf7-4eb1-b7ac-6c489e514d80', 'type': 'multiple_choice'}, 'type': 'choice', 'choice': {'id': 'A8PltOQn8LVm', 'label': '55-64'}}, {'field': {'id': 'ZNZyNVwMtw

# 2. Transform JSON to pandas.df and map dummy vars

### step by step

In [10]:
# LOAD ONE PERSON FROM THE JSON RESPONSE
one_person = responses["items"][0]

In [11]:
#transform ONE person's information to a dictionary using function one_entry_to_dictionary
dictionary_test = one_entry_to_dictionary(one_person)
print(dictionary_test)

{'id': 'lmykbgpt6c7v8zwulmykvg179i1hq716', 'submitted_at': '2020-05-26T03:23:05Z', 'gender': 'Woman', 'age': '55-64', 'ethnicity': 'Asian/Asian British', 'live_alone': 0, 'live_friends': 0, 'live_partner': 1, 'live_parents': 0, 'live_siblings': 0, 'live_otherfamily': 0, 'live_children': 1, 'live_other': 0, 'location': 'singapore', 'safety_level': 5, 'safety_change': 'Better', 'safety_why': 'worklife balance', 'mental_scale': 5, 'mental_problems_repetitive': 0, 'mental_problems_lonely': 0, 'mental_problems_stressed': 0, 'mental_problems_anxious': 1, 'mental_problems_worriedhealth': 0, 'mental_problems_scaredfuture': 0, 'mental_problems_relationship': 0, 'mental_problems_other': 0, 'mental_problems_none': 0, 'positive_impact_support_housework': 0, 'positive_impact_support_childcare': 0, 'positive_impact_connect': 0, 'positive_impact_leisure': 0, 'positive_impact_wfh': 0, 'positive_impact_other': 0, 'positive_impact_none': 0, 'work_situation': 'Yes, I work at home', 'financial_situation':

In [12]:
# transform dictionary to df 
df_test = pd.DataFrame(dictionary_test,index=[0])
df_test.head()

,id,submitted_at,gender,age,ethnicity,live_alone,live_friends,live_partner,live_parents,live_siblings,...,positive_impact_connect,positive_impact_leisure,positive_impact_wfh,positive_impact_other,positive_impact_none,work_situation,financial_situation,housework_amount,housework_change,testimonial
0,lmykbgpt6c7v8zwulmykvg179i1hq716,2020-05-26T03:23:05Z,Woman,55-64,Asian/Asian British,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",No impact - I feel the same about my financial...,We share it equally,I do more but it's manageable and sustainable,0


### apply this to entire JSON

In [13]:
def transform_all_to_df(responses):
    '''
    transforms the JSON response from the above query into a df with cols=vars, rows=individuals
    '''
    no_df_defined = True
    for item in responses["items"]:
        dictionary_transformation = one_entry_to_dictionary(item)
        df = pd.DataFrame(dictionary_transformation,index=[0])
        #crate dictionary and then append to it 
        if no_df_defined:
            df_all = df
            no_df_defined = False
        else: 
            df_all = df_all.append(df)
        df_all = df_all.reset_index(drop = True)
    return df_all

In [14]:
df_all = transform_all_to_df(responses)

te4mdfpy8xzj0q0aste4mc8ymac89pjn has entered in column other_work: I am a student, and will be starting to work from home in an internship soon. // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
gpx1pz49l087v5fbgpx1p7blxek6jl9j has entered in column other_work: Was a carer // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
9x0ba6cpkcchqth9vd9x0ba6cdsxp18z has entered in column other_work: I run my own business and am working all the time but not generating any revenue because of Covid. I also have a 7 month old baby. // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
j7wu1bgf0jh06mxu9hkwicj7wul45fgi has entered in column other_work: Unemployed due to child // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
j3how3ucddd42l75uj3how3unt6vj27w has entered in column other_work: In between jobs // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
8lg2mtsg3ala9xkk8lg2mdirml0bf04a has entered in column other_work: Maternity Leave // THIS ENTRY WAS DISCARDED AND SAVED AS OTHER
g3re0wvwmge256f6g3re3c09cv3hn7ej has en

In [15]:
df_all.head()

,id,submitted_at,gender,age,ethnicity,live_alone,live_friends,live_partner,live_parents,live_siblings,...,positive_impact_connect,positive_impact_leisure,positive_impact_wfh,positive_impact_other,positive_impact_none,work_situation,financial_situation,housework_amount,housework_change,testimonial
0,lmykbgpt6c7v8zwulmykvg179i1hq716,2020-05-26T03:23:05Z,Woman,55-64,Asian/Asian British,0,0,1,0,0,...,0,0,0,0,0,"Yes, I work at home",No impact - I feel the same about my financial...,We share it equally,I do more but it's manageable and sustainable,0
1,pjm4cz8u8arq3iiwfwpjm4wzxggqoqjd,2020-05-25T15:42:02Z,Woman,18-24,White,0,0,0,1,1,...,0,0,0,0,0,"Yes, but I had to reduce working hours because...",It has had a bit of a negative impact and has ...,We share it equally,I do more but it's manageable and sustainable,0
2,0fbf1s3x05pvign98jd0fbf1s3th3jxf,2020-05-25T15:34:36Z,Woman,18-24,White,1,0,0,0,0,...,0,0,0,0,0,"Yes, I work at home",It has had a bit of a negative impact and has ...,I take care of all the housework,I do more but it's manageable and sustainable,0
3,au8qsv21iddwsn6z8p9fau8qy07c39dq,2020-05-25T06:44:53Z,Man,18-24,White,0,0,0,1,1,...,0,0,0,0,0,"No, I don't work (or I am a student, or retired)",I feel better about my finances than I did before,I do less housework than others,I do more but it's manageable and sustainable,0
4,coqryi0o1s7fcoqrcd7s1zx02px19r7c,2020-05-23T11:34:52Z,Woman,18-24,White,0,0,0,1,0,...,0,0,0,0,0,"Yes, but I had to reduce working hours because...",I feel better about my finances than I did before,I do less housework than others,I haven't noticed a change,I know I had it very easy. The only « difficul...


In [16]:
len(df_all)

500

In [ ]:
df_all.columns

In [ ]:
# If you want to save the data 
"""
timestr = time.strftime("%Y%m%d%H%M")
file_name = "survey_data_" + timestr
print(file_name)

df_all.to_csv("../data/" + file_name + ".csv")

"""

# Aggregation for Graph 0 - Map 

In [ ]:
%load_ext autoreload
%autoreload 2
from map_graph.data_processing import merge_local_internat_dataframe

In [ ]:
postal_code = pd.read_csv("../data/postcode_uk.csv", index_col = 0)
#countries = pd.read_csv("https://raw.githubusercontent.com/rjcnrd/domestic_violence_covid-19/master/data/countries.csv")
countries = pd.read_csv("../data/countries.csv", index_col = 0)

In [ ]:
all_reports_df, not_in_map, uk_data, uk_data_county = merge_local_internat_dataframe(survey_df = df_all, postal_code_df = postal_code, countries_df = countries,  map_threshold = 1, postal_code_col = "location")

In [ ]:
all_reports_df.to_csv("../data/data_map_all_reports.csv")

## Check for the rows that are not integrated

In [ ]:
#Here are the values that are not integrated, by order
not_in_map.location.value_counts()

In [ ]:
#If you want to append new location, you should update the countries csv, one by one 
#country_name : name of the country as you see it in the survey
#Area : name of the country as it appears already in the survey. THIS WILL BE USED FOR A MERGE
#Area_name : name of the country as it should appear in the hover (the same name as area name)
"""
countries_new = countries.append({"latitude": 22.396428, 
                                  "longitude": 114.10949699999999, 
                                  "country_name" : "HKSAR",
                                  "area": "HONG KONG", 
                                  "area_name": "Hong Kong"}, ignore_index=True)
countries_new.to_csv("/Users/ameliemeurer/Documents/Amélie/HEC M2/07. Research Paper - UN Women/02. Code/domestic_violence_covid-19/data/countries.csv")
                                  """


# Aggregation for Graph 1. Scatter-Testimonial


In [19]:
%load_ext autoreload
%autoreload 2
from scatter_mental_health.data_processing import location_scatterplot, merge_testimonials, testimonial_treatment

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
#Create the location for all the dots
location_data = location_scatterplot(df_all, do_percentage = False, include_other=False, include_man = True, num_by_col=5)
location_data.head()

,gender,mental_health,order,x,y
0,Man,0,1,38,0
1,Man,1,1,44,0
2,Man,1,1,45,0
3,Man,2,1,50,0
4,Man,2,1,51,0


## Prepare Review by Giselle

In [21]:
df_graph_1_giselle = df_all[["submitted_at","id","testimonial"]]
    
#TAKE OUT TESTIMONIALS BEFORE WE CHANGED THE TEXT
df_graph_1_giselle = df_graph_1_giselle[(df_graph_1_giselle.submitted_at > "2020-05-07T00:00:00Z")==True]

# TAKE OUT NOS AND EMPTY 

df_graph_1_giselle = df_graph_1_giselle[(df_graph_1_giselle.testimonial!= 0) & ((df_graph_1_giselle.testimonial) != "NO")]

In [22]:
df_graph_1_giselle_selected_fields = df_graph_1_giselle[["id","testimonial"]]

In [23]:
# If you want to save the data 
'''
timestr = time.strftime("%Y%m%d%H%M")
file_name = "giselle_testimonials_feedback_requested" + timestr
print(file_name)

df_graph_1_giselle_selected_fields.to_csv(file_name + ".csv")
'''
# share this file and save results as giselle filled them in 



'\ntimestr = time.strftime("%Y%m%d%H%M")\nfile_name = "giselle_testimonials_feedback_requested" + timestr\nprint(file_name)\n\ndf_graph_1_giselle_selected_fields.to_csv(file_name + ".csv")\n'

## Import selected testimonials

In [24]:
all_testimonials = pd.read_excel("../data/giselle_testimonials_feedback_requested202005181636.xlsx")
selected_testimonials = all_testimonials.loc[all_testimonials["Going Live without email confirmation (Claire's selection)"].notna()]
selected_testimonials = selected_testimonials.rename(columns = {"Unnamed: 0" : "id", "Going Live without email confirmation (Claire's selection)":"selected_testimonial"})
selected_testimonials.head()

,id,testimonial,Provided their email adress,Claire Decision Yes / No,selected_testimonial,testimonial_giselle
3,m3c0j9m6utxx13402qc6jm3c0jm23p62,It’s very hard feeling that I need to make sur...,No,NaN,My family is in another country and I wish I c...,NaN
6,18qbxwx8tbi7m09o5u61ec18qbxwx8ih,I have a mental health condition and have been...,No,NaN,One thing that's negatively affected me...is n...,NaN
16,x0q8l0621w40aziuwx0q8lj5dxbz3suu,Whilst the situation has given me more time at...,No,NaN,Whilst the situation has given me more time at...,NaN
21,u9pomnn5f6huguktyfu9poqh5egjxsir,I feel that my mental health has been severely...,No,Y,My mental health has been severely affected by...,"""I feel that my mental health has been severel..."
38,t97s86p9h69rz0jcqt97soo8a57c0qe6,I am experiencing relatively extreme mood swin...,No,Y,I am experiencing relatively extreme mood swin...,"""I am experiencing relatively extreme mood swi..."


In [25]:
df_all = df_all.merge(selected_testimonials[["id", "selected_testimonial"]], left_on="id", right_on = "id", how = "left").fillna(0)

In [26]:
df_all.head()

,id,submitted_at,gender,age,ethnicity,live_alone,live_friends,live_partner,live_parents,live_siblings,...,positive_impact_leisure,positive_impact_wfh,positive_impact_other,positive_impact_none,work_situation,financial_situation,housework_amount,housework_change,testimonial,selected_testimonial
0,lmykbgpt6c7v8zwulmykvg179i1hq716,2020-05-26T03:23:05Z,Woman,55-64,Asian/Asian British,0,0,1,0,0,...,0,0,0,0,"Yes, I work at home",No impact - I feel the same about my financial...,We share it equally,I do more but it's manageable and sustainable,0,0
1,pjm4cz8u8arq3iiwfwpjm4wzxggqoqjd,2020-05-25T15:42:02Z,Woman,18-24,White,0,0,0,1,1,...,0,0,0,0,"Yes, but I had to reduce working hours because...",It has had a bit of a negative impact and has ...,We share it equally,I do more but it's manageable and sustainable,0,0
2,0fbf1s3x05pvign98jd0fbf1s3th3jxf,2020-05-25T15:34:36Z,Woman,18-24,White,1,0,0,0,0,...,0,0,0,0,"Yes, I work at home",It has had a bit of a negative impact and has ...,I take care of all the housework,I do more but it's manageable and sustainable,0,0
3,au8qsv21iddwsn6z8p9fau8qy07c39dq,2020-05-25T06:44:53Z,Man,18-24,White,0,0,0,1,1,...,0,0,0,0,"No, I don't work (or I am a student, or retired)",I feel better about my finances than I did before,I do less housework than others,I do more but it's manageable and sustainable,0,0
4,coqryi0o1s7fcoqrcd7s1zx02px19r7c,2020-05-23T11:34:52Z,Woman,18-24,White,0,0,0,1,0,...,0,0,0,0,"Yes, but I had to reduce working hours because...",I feel better about my finances than I did before,I do less housework than others,I haven't noticed a change,I know I had it very easy. The only « difficul...,0


## Prepare Text Wrapping

In [27]:
#Create the treatment for testimonials 
MAXIMUM_TOTAL_LENGTH = None #Maximum length of testimonial
MAXIMUM_LINE_LENGTH = 30 #Maximum length of the line on hover
MINIMUM_TOTAL_LENGTH = 10 #Minimum length of testimonial
testimonial_treated = testimonial_treatment(df_all[["gender", "mental_scale","selected_testimonial"]], 
                                            maximum_total_length = MAXIMUM_TOTAL_LENGTH, 
                                            maximum_line_length = MAXIMUM_LINE_LENGTH, 
                                            minimum_total_length = MINIMUM_TOTAL_LENGTH)
testimonial_treated.head()

../scatter_mental_health/data_processing.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
../scatter_mental_health/data_processing.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def merge_testimonials(location_data, testimonial_data, include_other=True):


,gender,mental_scale,selected_testimonial,testimonials_short,display_testimonial
0,Woman,5,0,0,0
1,Woman,3,0,0,0
2,Woman,3,0,0,0
3,Man,4,0,0,0
4,Woman,4,0,0,0


In [28]:
#Merges the 2 dataframes
data_scatterplot = merge_testimonials(location_data, testimonial_treated, include_other = False)
data_scatterplot.tail()

,gender,mental_health,order,x,y,testimonials_short,display_testimonial
487,Woman,5,0,30,11,0,0
488,Woman,5,0,31,11,0,0
489,Woman,5,0,32,11,0,0
490,Woman,5,0,33,11,0,0
491,Woman,5,0,34,11,0,0


In [29]:
data_scatterplot.to_csv("../data/data_scatterplot.csv")